# Step 1 Loading and Preprocessing the CIFAR-10 Dataset

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Data augmentation and normalization for the training set
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images
    transforms.RandomCrop(32, padding=4),  # Randomly crop images
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

# Only normalization for the validation/test set
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


# Step 2: Building a Simple CNN in PyTorch

In [2]:
import torch.nn as nn
import torch.nn.functional as F

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # Convolutional layer 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # Convolutional layer 2
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # Fully connected layer 1
        self.fc2 = nn.Linear(512, 10)  # Output layer for 10 classes (CIFAR-10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply conv1, ReLU, and pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply conv2, ReLU, and pooling
        x = x.view(-1, 64 * 8 * 8)  # Flatten the feature maps
        x = F.relu(self.fc1(x))  # Apply fully connected layer
        x = self.fc2(x)  # Output layer
        return x

# Instantiate the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Step 3: Training the CNN

In [3]:
# Training loop
for epoch in range(10):  # Train for 10 epochs
    running_loss = 0.0
    model.train()  # Set the model to training mode
    for images, labels in trainloader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.512399113848996
Epoch 2, Loss: 1.165593750641474
Epoch 3, Loss: 1.0143796120153363
Epoch 4, Loss: 0.9291546377532013
Epoch 5, Loss: 0.8578751852445285
Epoch 6, Loss: 0.8160848501698136
Epoch 7, Loss: 0.7710259857080172
Epoch 8, Loss: 0.7420307053324512
Epoch 9, Loss: 0.7169025774730746
Epoch 10, Loss: 0.6916019169951949


# Step 4: Evaluating the CNN on the Validation/Test Set

In [4]:
correct = 0
total = 0
model.eval()  # Set the model to evaluation mode (no backpropagation)
with torch.no_grad():  # Disable gradient calculation during evaluation
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')


Accuracy on the test set: 76.9%


# Step 5: Adding Regularization Techniques

In [5]:
# CNN with Dropout
class CNNWithDropout(nn.Module):
    def __init__(self):
        super(CNNWithDropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)  # Dropout layer with 50% dropout rate
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        return x

# Instantiate the model with dropout, loss function, and optimizer
model = CNNWithDropout()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [6]:
# Training loop
for epoch in range(10):  # Train for 10 epochs
    running_loss = 0.0
    model.train()  # Set the model to training mode
    for images, labels in trainloader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.6003673465355583
Epoch 2, Loss: 1.313198335640266
Epoch 3, Loss: 1.1802653616956433
Epoch 4, Loss: 1.1012834956883775
Epoch 5, Loss: 1.0411427121638033
Epoch 6, Loss: 1.005745127225471
Epoch 7, Loss: 0.967348417205274
Epoch 8, Loss: 0.9375700215854303
Epoch 9, Loss: 0.9099758375636147
Epoch 10, Loss: 0.8965840090418715


In [7]:
correct = 0
total = 0
model.eval()  # Set the model to evaluation mode (no backpropagation)
with torch.no_grad():  # Disable gradient calculation during evaluation
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')


Accuracy on the test set: 73.2%


# Step 6: Transfer Learning

In [8]:
from torchvision import models

# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Modify the last fully connected layer to match the 10 classes of CIFAR-10
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)


/Users/luisvalencia/Desktop/PythonPersonalProjects/mmlbook/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/luisvalencia/Desktop/PythonPersonalProjects/mmlbook/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# # Freeze all layers except the last layer
# for param in model.parameters():
#     param.requires_grad = False

# # Allow training of the final fully connected layer
# for param in model.fc.parameters():
#     param.requires_grad = True


In [10]:
# Use a lower learning rate for fine-tuning
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.0001)

# Fine-tuning loop
for epoch in range(10):
    running_loss = 0.0
    model.train()
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 2.1253554596925333
Epoch 2, Loss: 1.8731958626786156
Epoch 3, Loss: 1.8010877085768657
Epoch 4, Loss: 1.7627622253449677


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
model.eval()  # Set the model to evaluation mode (no backpropagation)
with torch.no_grad():  # Disable gradient calculation during evaluation
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')


Accuracy on the test set: 39.94%


# Implementing LeNet

## Step 1: Implementing LeNet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)  # Output: 32x32 -> 32x32
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)             # Output: 32x32 -> 28x28 -> 14x14

        # Update the input size for the first fully connected layer
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # Changed from 16 * 5 * 5 to 16 * 6 * 6
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        # Check the shape before flattening
        #print(x.shape)  # This will now output (batch_size, 16, 6, 6)
        
        x = x.view(x.size(0), -1)  # Automatically infers correct dimensions
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate LeNet
lenet_model = LeNet()
print(lenet_model)

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EnhancedLeNet(nn.Module):
    def __init__(self):
        super(EnhancedLeNet, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)  # Output: 32x32 -> 32x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)            # Output: 32x32 -> 28x28 -> 14x14

        # Fully Connected Layers
        self.fc1 = nn.Linear(64 * 6 * 6, 512)  # Adjusted for the output of conv2
        self.fc2 = nn.Linear(512, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        # Check the shape before flattening
        #print(x.shape)  # Debugging
        
        x = x.view(x.size(0), -1)  # Automatically infers correct dimensions
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the enhanced LeNet
lenet_model = EnhancedLeNet()


## Step 2: Training LeNet

In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data preparation (using CIFAR-10 as in the main chapter)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet_model.parameters(), lr=0.01, momentum=0.9)

# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    lenet_model.train()
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = lenet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(trainloader)}")


Epoch 1/10, Loss: 1.7920724437059954
Epoch 2/10, Loss: 1.3642825575740747
Epoch 3/10, Loss: 1.165209131091452
Epoch 4/10, Loss: 1.042144082924899
Epoch 5/10, Loss: 0.956881227929269
Epoch 6/10, Loss: 0.891823817435128
Epoch 7/10, Loss: 0.8280908525980952
Epoch 8/10, Loss: 0.7875347112679421
Epoch 9/10, Loss: 0.7547879245732446
Epoch 10/10, Loss: 0.7147584783527857


## Step 3: Evaluating LeNet on the Test Set

In [ ]:
correct = 0
total = 0
lenet_model.eval()
with torch.no_grad():
    for images, labels in testloader:
        outputs = lenet_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"LeNet Accuracy on CIFAR-10 test set: {100 * correct / total:.2f}%")


LeNet Accuracy on CIFAR-10 test set: 73.13%
